### Utilities

In [ ]:
!pip install -q lucid>=0.2.3

In [ ]:
import numpy as np
import ctypes.util
from lucid.misc.gl.glcontext import create_opengl_context

# Now it's safe to import OpenGL and EGL functions
import OpenGL.GL as gl
from OpenGL.GLU import *

# create_opengl_context() creates GL context that is attached to an
# offscreen surface of specified size. Note that rendering to buffers
# of different size and format is still possible with OpenGL Framebuffers.
#
# Users are expected to directly use EGL calls in case more advanced
# context management is required.
WIDTH, HEIGHT = 800,600
create_opengl_context((WIDTH, HEIGHT))
    
# OpenGL context is available here.

print(gl.glGetString(gl.GL_VERSION))
print(gl.glGetString(gl.GL_VENDOR)) 
#print(gl.glGetString(gl.GL_EXTENSIONS))

b'4.4.0 NVIDIA 000.00'
b'NVIDIA Corporation'


In [ ]:
# Custom Functions

def addVertex(x, y):
    gl.glVertex2f(x/(WIDTH/2), y/(HEIGHT/2))

def addColor(r, g, b):
    gl.glColor3f(r/255, g/255, b/255)


## Draw ID

### Find Zone

In [ ]:
def findZone(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1

    if abs(dx) > abs(dy):
        if dx >= 0 and dy >= 0:
            return 0
        elif dx <= 0 and dy >= 0:
            return 3
        elif dx <= 0 and dy <= 0:
            return 4
        elif dx >= 0 and dy <= 0:
            return 7
    else:
        if dx >= 0 and dy >= 0:
            return 1
        elif dx <= 0 and dy >= 0:
            return 2
        elif dx <= 0 and dy <= 0:
            return 5
        elif dx >= 0 and dy <= 0:
            return 6


### Convert to Zone Zero

In [ ]:
def convertToZoneZero(x, y, zone):
    if zone == 0:
        return x, y
    elif zone == 1:
        return y, x
    elif zone == 2:
        return -y, x
    elif zone == 3:
        return -x, y
    elif zone == 4:
        return -x, -y
    elif zone == 5:
        return -y, -x
    elif zone == 6:
        return y, -x
    elif zone == 7:
        return x, -y


### Convert back to Original Zone

In [ ]:
def convertToOriginalZone(x, y, zone):
    if zone == 0:
        return x, y
    elif zone == 1:
        return y, x
    elif zone == 2:
        return y, -x
    elif zone == 3:
        return -x, y
    elif zone == 4:
        return -x, -y
    elif zone == 5:
        return -y, -x
    elif zone == 6:
        return -y, x
    elif zone == 7:
        return x, -y


### Draw Line using Mid Point Algorithm

In [ ]:
def drawLine(x1, y1, x2, y2):
    zone = findZone(x1, y1, x2, y2)
    x1, y1 = convertToOriginalZone(x1, y1, zone)
    x2, y2 = convertToOriginalZone(x2, y2, zone)
  
    dx = x2 - x1
    dy = y2 - y1

    d = 2 * dy - dx
    dE = 2 * dy
    dNE = 2 * (dy - dx)

    x = x1
    y = y1
  
    addColor(0, 255, 195)
    gl.glPointSize(20)
    gl.glBegin(gl.GL_POINTS)

    while x < x2:
        x += 1
        if d < 0:
            d += dE
        else:
            y += 1
            d += dNE
    
        xN, yN = convertToZoneZero(x, y, zone)
        addVertex(xN, yN)

    gl.glEnd()


### Draw digits

In [ ]:
def drawDigit(num, direction = -1):
    x = [80, 160, 320, -80, -160, -320]
    y = [60, 120, 240, -60, -120, -240]

    # 0
    if num == 0:
        drawLine(x[0]*direction, y[2], x[2]*direction, y[2])
        drawLine(x[0]*direction, y[2], x[0]*direction, y[5])
        drawLine(x[2]*direction, y[2], x[2]*direction, y[5])
        drawLine(x[0]*direction, y[5], x[2]*direction, y[5])

    # 1
    elif num == 1:
        if direction == 1:
            drawLine(x[2], y[2], x[2], y[5])
            drawLine(x[0], y[1], x[2], y[2])
        else:
            drawLine(x[3], y[2], x[3], y[5])
            drawLine(x[5], y[1], x[3], y[2])

    # 2
    elif num == 2:
        drawLine(x[0]*direction, y[2], x[2]*direction, y[2])
        drawLine(x[2]*direction, 0, x[0]*direction, 0)
        drawLine(x[0]*direction, y[5], x[2]*direction, y[5])

        if direction == 1:
            drawLine(x[0], 0, x[0], y[5])
            drawLine(x[2], y[2], x[2], 0)

        else:
            drawLine(x[3], y[2], x[3], 0)
            drawLine(x[5], 0, x[5], y[5])

    # 3
    elif num == 3:
        drawLine(x[0]*direction, y[2], x[2]*direction, y[2])
        drawLine(x[0]*direction, y[5], x[2]*direction, y[5])
        drawLine(x[0]*direction, 0, x[2]*direction, 0)

        if direction == 1:
            drawLine(x[2], y[2], x[2], 0)
            drawLine(x[2], 0, x[2], y[5])
        else:
            drawLine(x[3], y[2], x[3], 0)
            drawLine(x[3], 0, x[3], y[5])

    # 4
    elif num == 4:
        drawLine(x[2]*direction, 0, x[0]*direction, 0)

        if direction == 1:
            drawLine(x[2], y[2], x[2], y[5])
            drawLine(x[0], 0, x[2], y[2])
        else:
            drawLine(x[3], y[2], x[3], y[5])
            drawLine(x[5], 0, x[3], y[2])

    # 5
    elif num == 5:
        drawLine(x[0]*direction, y[2], x[2]*direction, y[2])
        drawLine(x[2]*direction, 0, x[0]*direction, 0)
        drawLine(x[0]*direction, y[5], x[2]*direction, y[5])

        if direction == 1:
            drawLine(x[0], 0, x[0], y[2])
            drawLine(x[2], 0, x[2], y[5])
        else:
            drawLine(x[3], 0, x[3], y[5])
            drawLine(x[5], 0, x[5], y[2])

    # 6
    elif num == 6:
        drawLine(x[0]*direction, y[2], x[2]*direction, y[2])
        drawLine(x[2]*direction, 0, x[0]*direction, 0)
        drawLine(x[0]*direction, y[5], x[2]*direction, y[5])

        if direction == 1:
            drawLine(x[2], 0, x[2], y[5])
            drawLine(x[0], 0, x[0], y[2])
            drawLine(x[0], 0, x[0], y[5])
        else:
            drawLine(x[3], 0, x[3], y[5])
            drawLine(x[5], 0, x[5], y[2])
            drawLine(x[5], 0, x[5], y[5])

    # 7
    elif num == 7:
        drawLine(x[0]*direction, y[2], x[2]*direction, y[2])

        if direction == 1:
            drawLine(x[0], y[5], x[2], y[2])
        else:
            drawLine(x[5], y[5], x[3], y[2])

    # 8
    elif num == 8:
        drawLine(x[0]*direction, y[2], x[2]*direction, y[2])
        drawLine(x[0]*direction, y[2], x[0]*direction, y[5])
        drawLine(x[2]*direction, y[2], x[2]*direction, y[5])
        drawLine(x[0]*direction, y[5], x[2]*direction, y[5])
        drawLine(x[0]*direction, 0, x[2]*direction, 0)

    # 9
    elif num == 9:
        drawLine(x[0]*direction, y[2], x[2]*direction, y[2])
        drawLine(x[2]*direction, 0, x[0]*direction, 0)
        drawLine(x[0]*direction, y[5], x[2]*direction, y[5])

        if direction == 1:
            drawLine(x[2], y[2], x[2], 0)
            drawLine(x[2], 0, x[2], y[5])
            drawLine(x[0], 0, x[0], y[2])
        else:
            drawLine(x[3], y[2], x[3], 0)
            drawLine(x[3], 0, x[3], y[5])
            drawLine(x[5], 0, x[5], y[2])


### Driver Code

In [ ]:
from IPython.display import display
from PIL import Image

gl.glClear(gl.GL_COLOR_BUFFER_BIT)

stdID = input()

if len(stdID) == 8:
    drawDigit(int(stdID[-2]))
    drawDigit(int(stdID[-1]), 1) # Direction = 1 means right digit

    img_buf = gl.glReadPixelsub(0, 0, WIDTH, HEIGHT, gl.GL_RGB, gl.GL_UNSIGNED_BYTE)
    img = np.frombuffer(img_buf,np.uint8).reshape(HEIGHT, WIDTH, 3)[::-1]
    display(Image.fromarray(img,'RGB'))

else:
    print("Wrong ID")
